In [1]:
from bn256 import *
import random
from copy import copy
from functools import reduce
import time

In [2]:
def timer(func):
    def _func(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        t = time.clock() - t
        print('{0}: time elapsed {1:.8f} s.' .format(func.__name__, t))
        return res

    return _func

In [3]:
def hamminge_distance(s1, s2):
    l = len(s1)
    assert l == len(s2), "Undefined for sequences of unequal length."
    return sum(el1 != el2 for el1, el2 in zip(s1, s2)) / l

In [4]:
a = (-1, 1)
def gen_vector(size):
    return tuple(random.choice(a) for i in range(size))

In [5]:
v1, v2 = gen_vector(16), gen_vector(16)

In [6]:
print(v1, v2, sep="\n")

(-1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1)
(-1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1)


In [7]:
def inverse_G1(curve):
    inv = copy(curve)
    inv.y = inv.y.additive_inverse()
    return inv

In [8]:
def inverse_G2(twist):
    inv = copy(twist)
    inv.negate()
    return inv

In [9]:
def transfer_vec_to_G1(vec):
    return tuple(inverse_G1(curve_G) if p == -1 else curve_G for p in vec)

In [10]:
def transfer_vec_to_G2(vec):
    return tuple(inverse_G2(twist_G) if p == -1 else twist_G for p in vec)

In [11]:
def gen_keys(size):
    return tuple(random.randrange(2, order) for i in range(size))

In [12]:
v1points = transfer_vec_to_G2(v1)
v2points = transfer_vec_to_G1(v2)

In [13]:
v1points

(((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((2116796163654258025501177006657054

In [14]:
v2points

((1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781))

In [15]:
h1, h2 = random.randrange(2, order), random.randrange(2, order)
H1, H2 = curve_G.scalar_mul(h1), twist_G.scalar_mul(h2)

In [16]:
h1, h2

(52834071928938962065135864076752246649196307995624320620024694060596447079009,
 48574895824709254458403476493974420248462101857336862733496373183820927652450)

In [17]:
@timer
def master_key_generation():
    s, t = gen_keys(16), gen_keys(16)
    u, v = gen_keys(18), gen_keys(18)
    gen1_h = tuple(point_add(curve_G.scalar_mul(si), H1.scalar_mul(ti)) for si, ti in zip(s, t))
    gen2_h = tuple(point_add(twist_G.scalar_mul(ui), H2.scalar_mul(vi)) for ui, vi in zip(u, v))
    return gen1_h, s, t, gen2_h, u, v

In [18]:
gen1_h, s, t, gen2_h, u, v = master_key_generation()

master_key_generation: time elapsed 16.81945866 s.


In [19]:
gen1_h, s, t, gen2_h, u, v

(((29740349647444273041308904794403414932745845538660681532768818487324693929869, 44452912330826396443226101701070739023924889764237370031542025013175778048490),
  (22456308857833899581480532508342932417849462717353850716449037965717634156794, 47459172078241221871813467250724295610953845348845848370250616879090765844311),
  (212536773423944155513436703112234502022852695317933511008788628697437608495, 44959991301772479582811553985644389272615425361277536825312949098865075896767),
  (36889763821187892819462506382207069362271686797958690747067601841678532192357, 49726369053847311800538434597733885702066926059351328589461996328011750483420),
  (26462026840835982119729903003788556457324658440529318924918300507132390509803, 57768050271069490611297541543625093583859065519842894409239585607339785918631),
  (18618667576279295571464956061922819489658035826526302848910129903671937732730, 13933055670019071968377238875671642480952342453280892057658934616736930724867),
  (273941684340844189973898653

In [20]:
@timer
def decryption_key_generation(gen2_h, s, t, v1points):
    n = len(v1points)
    
    r0 = random.randrange(2, order)
    reg_template = [twist_G.scalar_mul(r0), H2.scalar_mul(r0)]
    
    d03 = v1points[0].scalar_mul(s[0])
    d04 = v1points[0].scalar_mul(t[0])
    for i in range(1, n):
        d03 = point_add(d03, v1points[i].scalar_mul(s[i]))
        d04 = point_add(d04, v1points[i].scalar_mul(t[i]))
    reg_template.append(point_add(inverse_G2(d03), gen2_h[0].scalar_mul(r0)))
    reg_template.append(point_add(inverse_G2(d04), gen2_h[1].scalar_mul(r0)))

    for i in range(n):
        reg_template.append(point_add(v1points[i], gen2_h[i + 2].scalar_mul(r0)))
    
    return tuple(reg_template)

In [21]:
DK = decryption_key_generation(gen2_h, s, t, v1points)

decryption_key_generation: time elapsed 14.42778750 s.


In [22]:
len(DK)

20

In [23]:
DK

(((35426270462353736404323392156368437693036770807080218931580242583194475992956,57326040881044452951926874764667890325541439356987649126567532238224930876725), (20569261623352873037877125828114463339410921610640069070328007274516653645393,34946784487778813548839432778123617870478867557711931414604745093467182190838)),
 ((1895198626781875525732719634949808880408481414254374715184475190638226688395,13208772670191843656333042900615774575130620393730717535951261239688170668788), (34366867432896186663845691256752991414102363502297230816424652108891679634561,46569882563589791442843482837398036256898917610116181834618735752593688233040)),
 ((50784962549129042550632658965462078365353438426290738952111655907346900544272,59933666373954281020995113400533300211617304556188777559778045607159050901215), (12294470007230030777581018788049942932227130463101508723508131360160038030779,19423033609558590551046076892790239425158903758304802161590722066288727907863)),
 ((53135967491455648837084903592518103

In [24]:
@timer
def encryption_authentication(gen1_h, u, v, v2points):
    n = len(v1points)
    
    r0 = random.randrange(2, order)
    auth_template = [curve_G.scalar_mul(0), curve_G.scalar_mul(0), curve_G.scalar_mul(r0), H1.scalar_mul(r0)]
    
    for ci, h in zip(v2points, gen1_h):
        auth_template.append(point_add(ci, h.scalar_mul(r0)))
        
    c01 = auth_template[2].scalar_mul(u[0])
    c02 = auth_template[2].scalar_mul(v[0])
    for i in range(1, n + 2):
        c01 = point_add(c01, auth_template[i + 2].scalar_mul(u[i]))
        c02 = point_add(c02, auth_template[i + 2].scalar_mul(v[i]))
    
    auth_template[0] = point_add(auth_template[0], inverse_G1(c01))
    auth_template[1] = point_add(auth_template[1], inverse_G1(c02))
    
    return tuple(auth_template)

In [25]:
CT = encryption_authentication(gen1_h, u, v, v2points)

encryption_authentication: time elapsed 3.97646710 s.


In [26]:
len(CT)

20

In [27]:
CT

((54471241186726578047303389153117871500263439254150638399788255928535153298872, 3531835323000606252464096318935238631909078095347540693995233431305871666057),
 (17109079722216118815062911279877944063803724504906272575397271210216788361382, 48577692138459911368701613348700293306268272542563820153526281834078977759165),
 (19582344241330494646012674337353202316182102320105328469456645996243661682566, 34351289592770537037798909505019305082303799845244781503797345181099470970209),
 (23445464532431132659883248066075392549869055203111610853061482058097896684677, 7859336147396817804457870347752620842887237592234359916816062595777069625839),
 (1842653282462152442051597585319648027208107280350164111529519249648249863077, 56749313879838609936042913666197748682708433524887122178314600774649483237266),
 (48519443065685568359655471753779799430598218265058150606114610510422677644127, 9356626014538342866401611096967029666427046822771395264063706336537736509332),
 (294193256744368188839211617610975874

In [28]:
gt = optimal_ate(twist_G, curve_G)
logarithmic_table = [gt.exp(i) for i in range(16, -1, -2)]
gtm = optimal_ate(twist_G, inverse_G1(curve_G))
logarithmic_table.extend(gtm.exp(i) for i in range(2, 17, 2))

In [29]:
for i, el in enumerate(logarithmic_table):
    print(i, ": ", el)

0 :  (((15429308596100125345042025182838740585933899260990097211943232471185708209892,21036769640615344552925704451163498369215862996620789540674973406371475898801),(34643419400173764267828846220737244479195263121357284576095582127626220190690,39217953906512156784965537049391267021797863115575109425638059426537342418239),(29362508885087302295248031767237294697462853748363034881398247042502825752042,33065606296478674290051087119664030123566328954069047137872363079287669349803)),((44150688783494038190860128993786071429009665932812059212008957591291015396555,25855485444396824150330723346354416914393823131049522641950650987664836921571),(14355433543029713763013415741005074744216778912056694539583678910055649667776,46280493583452157638432486289407345007584489845082308921018418732775447515998),(35438949274284069241468591891906709017152328378869050685915198888251448805767,45559280046590109824720435394540600218774466880098159450808971756061223256757)))
1 :  (((597783055027885551703832584052683

In [30]:
@timer
def decryption_authentication(DK, CT):
    d = optimal_ate(DK[0], CT[0])
    for i in range(1, 20):
        d = d.mul(optimal_ate(DK[i], CT[i]))
    return logarithmic_table.index(d)

In [31]:
a = decryption_authentication(DK, CT)

decryption_authentication: time elapsed 16.89285040 s.


In [32]:
a

6

In [33]:
a / 16

0.375

In [34]:
hamminge_distance(v1, v2)

0.375